In [1]:
import numpy as np
# import scipy.stats as ss
# from scipy.spatial import distance_matrix
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import SpectralClustering
from sklearn.metrics.cluster import adjusted_rand_score
from tqdm.notebook import tqdm
from collections import defaultdict
from multiprocessing import Pool
import anndata
import scanpy

In [2]:
INPUT_PATH="/mnt/geofflab/SNP_barcoding/Lvar_annotations_v3_Jan2021/transcripts.csv"
COUNTS_PATH="/mnt/geofflab/SNP_barcoding/LV_counts_10hpf.csv"

In [3]:
heterozygosity = 0.001 # This is the rate of SNPs per base
alphabet = {"A":0, "C":1, "G":2, "T":3}

# genotypes where chromosome order matters (used internally during generation)
genotypes_internal = {
    "AA":0, "AC":1, "AG":2, "AT":3, 
    "CA":4, "CC":5, "CG":6, "CT":7,
    "GA":8, "GC":9, "GG":10, "GT":11, 
    "TA":12, "TC":13, "TG":14, "TT":15,
}

# genotypes where chromosome order doesn't matter (what we see)
genotypes_external = {
"AA":0, "AC":1, "AG":2, "AT":3, 
"CA":1, "CC":4, "CG":5, "CT":6,
"GA":2, "GC":4, "GG":7, "GT":8, 
"TA":3, "TC":6, "TG":8, "TT":9,
}

In [16]:
def sample_dropout_profile(args):
    """
    Sample a dropout profile from single-cell transcript read count df, and 
    introduce dropout over the snp_profile according to the dropout profile.
    """
    global adata_counts
    snps = args[0].copy()
    loci_map = args[1]
    
    np.random.seed()
    
    # Sample a cell's counts
    sampled_dprofile = np.random.choice(adata_counts.obs.index)

    for gene in adata_counts.var.index:
        count = int(adata_counts[sampled_dprofile, gene].X)

        # If we have no counts, then we have can't detect the genotype
        if count <= 1:
            for locus in loci_map[gene]:
                snps[locus] = -1 # -1 genotype -> undetected
        else:
            # Otherwise compute the probability of only seeing one of the chromosomes, or both
            prob_one = 0.5**count
            prob_both = 1 - 2*prob_one

            # Now choose whether we only see the first chromosome, second chromosome, or both
            seen = np.random.choice([0, 1, 2], p=[prob_one, prob_one, prob_both])

            if seen == 0:
                # Convert internal to external genotypes on the first chromosome
                first_mapping = {0:0, 1:0, 2:0, 3:0,
                                 4:4, 5:4, 6:4, 7:4,
                                 8:7, 9:7, 10:7, 11:7,
                                 12:9, 13:9, 14:9, 15:9}

                # Convert the SNP read to it's value on one of the chromosomes
                for snp_loc in loci_map[gene]:
                    snps[snp_loc] = first_mapping[snps[snp_loc]]

            elif seen == 1:
                # Convert internal to external genotypes on the first chromosome
                second_mapping = {0:0, 1:4, 2:7, 3:9,
                                 4:0, 5:4, 6:7, 7:9,
                                 8:0, 9:4, 10:7, 11:9,
                                 12:0, 13:4, 14:7, 15:9}

                # Convert the SNP read to it's value on one of the chromosomes
                for snp_loc in loci_map[gene]:
                    snps[snp_loc] = second_mapping[snps[snp_loc]]

            else:

                # Convert internal to external genotypes on both chromosomes
                both_mapping = {0:0, 1:1, 2:2, 3:3,
                                 4:1, 5:4, 6:5, 7:6,
                                 8:2, 9:4, 10:7, 11:8,
                                 12:3, 13:6, 14:8, 15:9}

                # Convert the SNP read to it's value on one of the chromosomes
                for snp_loc in loci_map[gene]:
                    snps[snp_loc] = both_mapping[snps[snp_loc]]

    return snps

class EmbryoSequenceSimulator:
    def __init__(self, input_df, n_embryos):
        
        # pd data frame
        self.input_df = input_df # cols: gene,chromosome,start,end,direction,sequence
        self.n_embryos = n_embryos
        
        # Number of cells that can be harvested from embryo
        self.avg_harvested_cells = 20
        self.sd_harvested_cells = 3
        self.alphabet = {"A":0, "C":1, "G":2, "T":3}
        
        # Global parental chromatid maps
        self.m0_chromatid = None
        self.m1_chromatid = None
        self.p0_chromatid = None
        self.p1_chromatid = None
        
    def global_seeding(self, heterozygosity):
        """
        Uniform seeding of SNP locations for each gene with frequency according to heterozygosity.
        Parental chromatid seeding (2 chromatids from mother, 2 chromatids from father)
        """
        
        # All the SNPs present in the parents
        snp_map_overall = None
        
        # Setting parental chromatid maps
        for parent in ["m0","m1","p0","p1"]:
            
            # Create a profile of SNPs for each parent chromosome
            snp_map = defaultdict(dict)
            sequence_map = defaultdict(dict)
            
            for idx,transcript_entry in self.input_df.iterrows():
                gene,chrom,start,end,direction,sequence = transcript_entry
                N = len(sequence)
                num_snps = np.random.binomial(N, p=heterozygosity)
                snp_positions = list(np.sort(random.sample(range(N),num_snps)))
                snp_map[chrom][gene] = snp_positions
                sequence_map[chrom][gene] = sequence
                
            # Add the snp map we just made to the global collection of SNPs
            if snp_map_overall is None:
                snp_map_overall = snp_map
            else:
                for idx,transcript_entry in self.input_df.iterrows():
                    gene,chrom,start,end,direction,sequence = transcript_entry
                    snp_map_overall[chrom][gene].extend(snp_map[chrom][gene])
            
            chromatid = defaultdict(dict)
            # If we have a SNP, this is the probability of going reference to 
            transition_probs = np.array([[0, 1/3, 1/3, 1/3],
                                         [1/3, 0, 1/3, 1/3],
                                         [1/3, 1/3, 0, 1/3],
                                         [1/3, 1/3, 1/3, 0]])
            
            for chrom,genes in snp_map.items():            
                for gene,snp_positions in genes.items():
                    sequence_list = [char for char in sequence_map[chrom][gene]]
                    snp_profile = {locus:np.random.choice(list(self.alphabet.keys()), 
                                                          p=transition_probs[alphabet[sequence_list[locus]]]) for locus in snp_positions}
                    
                    for locus,char in snp_profile.items():
                        sequence_list[locus] = char
                    
                    chromatid[chrom][gene] = "".join(sequence_list)
                
            # Assign seeded parental chromatid to its class field (global over all embryos)
            if parent == "m0":
                self.m0_chromatid = chromatid
            elif parent == "m1":
                self.m1_chromatid = chromatid
            elif parent == "p0":
                self.p0_chromatid = chromatid
            elif parent == "p1":
                self.p1_chromatid = chromatid
                
        return snp_map_overall
    
    def embryo_population_simulation(self):
        """
        Simulating SNP profile for each embryo across all transcript entries in input_df.
        For this simulator, not explicitly generating scRNA-seq reads for each embryo (reduces space)
        """

        # map: idx of transcript entry -> list of snp positions
        snp_map = self.global_seeding(heterozygosity)
        true_clusters = []
        cluster_label = 0
        
        population_profile = [] # The variant calls for the cells
        
        p = Pool(NUM_PROC)
        
        for embryo in range(self.n_embryos):
            
            # Each entry a list of genotypes for an embryo across all transcripts
            snp_profile,gene_locus_map = self.determine_embryo_genotype(snp_map)

            # Harvest some number of cells, which will theoretically have identical SNP profiles
            n_harvested_cells = 16
            
            # Parallelize making the cells
            new_cells = list(tqdm(p.imap(sample_dropout_profile, 
                                    zip([snp_profile]*n_harvested_cells, 
                                        [gene_locus_map]*n_harvested_cells)), 
                                  total=n_harvested_cells, desc=f'Embryo {embryo}'))
                
            population_profile.extend(new_cells)

            
            # Mark the cluster of the cells we just created
            for _ in range(n_harvested_cells):
                true_clusters.append(cluster_label)
            
            cluster_label += 1
            
        p.close()
                
        return np.array(population_profile),true_clusters
        
    
    def determine_embryo_genotype(self, snp_map):
        """
        Simulate array containing genotypes over all transcripts 
        (concatenated into single list)
        """
                
        # state (genotype) across all transcripts
        snp_genotypes,idx = [],0
        gene_locus_map = defaultdict(list)
                
        # map: idx of transcript entry -> sequence after homologous recombination
        m_chromatid,p_chromatid = self.simulate_embryo_sequence()
        for chrom,genes in snp_map.items():
            for gene,snp_positions in genes.items():                
                m_sequence = m_chromatid[chrom][gene]
                p_sequence = p_chromatid[chrom][gene]
                for locus in snp_positions:
                    snp_genotype = "".join([m_sequence[locus],p_sequence[locus]])
                    snp_genotypes.append(genotypes_internal[snp_genotype])
                    gene_locus_map[gene].append(idx)
                    idx += 1
                
        return snp_genotypes,gene_locus_map
                
    def simulate_embryo_sequence(self):
        """
        Given seeded parental chromatids and a map of SNP locations for each transcript,
        simulate embryo sequences
        """
        
        updated_m0,updated_m1 = defaultdict(dict),defaultdict(dict)
        updated_p0,updated_p1 = defaultdict(dict),defaultdict(dict)
        
        for parent in ["mother","father"]:
            
            # Consider homologous recombination with respect to each chromosome
            for chr_num in np.sort(np.unique(df["chromosome"])):
                
                # Subset df for specific chr, sorted by start position 
                chr_df = self.input_df[self.input_df["chromosome"] == chr_num].sort_values(by=["start"])
                
                # Chiasma point chosen over range of transcripts reads
                transcript_range = range(chr_df["start"].iloc[0], chr_df["start"].iloc[-1]+len(chr_df["sequence"].iloc[-1])) 
                chiasma = random.choice(transcript_range)
                chiasma_flag = 1 # 1 means chiasma has not been reached; 0 means passed chiasma
                for idx,transcript_entry in chr_df.iterrows():
                    if parent == "mother":
                        chiasma_flag = self.homologous_recombination(
                            transcript_entry = transcript_entry,
                            updated_c0 = updated_m0,
                            updated_c1 = updated_m1,
                            c0 = self.m0_chromatid,
                            c1 = self.m1_chromatid,
                            chiasma = chiasma,
                            flag = chiasma_flag
                        )
                    else:
                        chiasma_flag = self.homologous_recombination(
                            transcript_entry = transcript_entry,
                            updated_c0 = updated_p0,
                            updated_c1 = updated_p1,
                            c0 = self.p0_chromatid,
                            c1 = self.p1_chromatid,
                            chiasma = chiasma,
                            flag = chiasma_flag
                        )

        m_chromatid = self.chromosome_inheritance(updated_m0, updated_m1)
        p_chromatid = self.chromosome_inheritance(updated_p0, updated_p1)
        
        return m_chromatid,p_chromatid
    
    def homologous_recombination(self,transcript_entry,updated_c0,updated_c1,c0,c1,chiasma,flag):
        """
        for a given transcript entry, simulate process of homologous recombination, given
        a uniformly selected chiasma point. 
        """
        
        gene,chrom,start,end,direction,sequence = transcript_entry
        
        # 1 means chiasma has not been reached; 0 means passed chiasma
        if flag <= start and flag == 1:
            flag = 0
            
            # Case where chiasma point lies within a gene
            if chiasma in range(start,start+len(sequence)):
                updated_c0[chrom][gene] = c0[chrom][gene][:chiasma]+c1[chrom][gene][chiasma:]
                updated_c1[chrom][gene] = c1[chrom][gene][:chiasma]+c0[chrom][gene][chiasma:]
                return flag

        if flag == 1: # Before chiasma point has been reached
            updated_c0[chrom][gene] = c0[chrom][gene]
            updated_c1[chrom][gene] = c1[chrom][gene]
        else: # After chiasma point has been reached
            updated_c0[chrom][gene] = c1[chrom][gene]
            updated_c1[chrom][gene] = c0[chrom][gene]
        
        return flag
    
    def chromosome_inheritance(self,c0,c1):
        """
        Given homologous chromosomes, randomly select one to be inherited by embryo
        """
        
        chromatid = defaultdict(dict)
        for chrom in c0:
            chromatid[chrom] = c0[chrom] if random.random() <= 0.5 else c1[chrom]
        
        return chromatid
                  
    

/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Initialize Data

In [65]:
N_EMBRYOS = 40
NUM_PROC = 50

/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [66]:
df = pd.read_csv(INPUT_PATH)
df = df.dropna()

# Read the full counts without downsampling
count_df_full = pd.read_csv(COUNTS_PATH)
count_df_full = count_df_full[count_df_full["Lv_name"] != "LVA_m28240.t1"]

# Convert to anndata so we can downsample with scanpy
adata_counts_full = anndata.AnnData(X=count_df_full.iloc[:, 2:].to_numpy().T,
                               obs=pd.DataFrame(index=count_df_full.iloc[:, 2:].columns),
                               var=pd.DataFrame(index=count_df_full['Lv_name']))

In [67]:
# Dowsample all sequences to max 150 bp
for ind in tqdm(df.index):
    start = df.loc[ind, 'start']
    end = df.loc[ind, 'end']
    sequence = df.loc[ind, 'sequence']
    
    # If a sequence is longer than 150, trim it
    if np.absolute(end-start) > 150:
        df.loc[ind, 'end'] = start + 150
        df.loc[ind, 'sequence'] = sequence[:150]

/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/29836 [00:00<?, ?it/s]

## Simulate Test Data

We need to simulate variant call matrices for a mix of downsampling levels.

In [68]:
# Get the avg and total UMI for the base counts matrix
avg_umi_full = np.average(np.sum(adata_counts_full.X, axis=1))
total_counts_full = np.sum(adata_counts_full.X)

In [ ]:
# The fraction of counts we should retain for each run
DS_LEVELS = [250, 750, 1250, 1750, np.floor(avg_umi_full)]

variant_mats = []
true_clusterings = []
avg_umi = []

# Downsample the counts matrix to each level and generate data
for ds_level in DS_LEVELS:  
    counts_target = adata_counts_full.shape[0]*ds_level
    
    # Downsample the matrix
    adata_counts = adata_counts_full.copy()
    scanpy.pp.downsample_counts(adata_counts, total_counts=counts_target)
    avg_umi.append(np.average(np.sum(adata_counts.X, axis=1)))
        
    # Generate the embryos
    simulator = EmbryoSequenceSimulator(df, N_EMBRYOS)
    X,true_clusters = simulator.embryo_population_simulation()
    
    variant_mats.append(X)
    true_clusterings.append(true_clusters)

Embryo 0:   0%|          | 0/20 [00:00<?, ?it/s]

Embryo 1:   0%|          | 0/23 [00:00<?, ?it/s]

Embryo 2:   0%|          | 0/26 [00:00<?, ?it/s]

Embryo 3:   0%|          | 0/22 [00:00<?, ?it/s]

Embryo 4:   0%|          | 0/20 [00:00<?, ?it/s]

...

Embryo 16:   0%|          | 0/21 [00:00<?, ?it/s]

Embryo 17:   0%|          | 0/17 [00:00<?, ?it/s]

Embryo 18:   0%|          | 0/17 [00:00<?, ?it/s]

Embryo 19:   0%|          | 0/16 [00:00<?, ?it/s]

Embryo 20:   0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
def adjustedHamming(g0, g1):
# Computes a hamming distance normalized by where we have
# data. ie. entries are not -1.
    if len(g0) != len(g1):
        raise ValueError("Sequence lengths unequal.")
    else:
        hd,adjlen = 0, 0
        for i, j in zip(g0, g1):
            if i == -1 or j == -1: 
                continue
            
            adjlen += 1
            
            if i != j: 
                hd += 1
                
        if adjlen == 0:
            return np.inf
        else:
            return hd/adjlen

def compute_row_ham_dists(i):
# Compute a row of hamming dists
    row_hd = np.zeros(N-i)
    
    for j in range(i, N):
        hd = adjustedHamming(X[i,:], X[j,:])
        row_hd[j-i] = hd
        
    return row_hd

ham_dists_ds = []

# Compute the hamming dist matrices for each of the downsamples
for i in range(len(variant_mats)):
    X = variant_mats[i]
    N = X.shape[0]

    with Pool(NUM_PROC) as p:
        ham_dists = list(tqdm(p.imap(compute_row_ham_dists, range(N)), total=N))

    hd = np.zeros((N, N))

    for i in range(N):
        hd[i, i:] = ham_dists[i]

    hd = hd + hd.T
    
    ham_dists_ds.append(hd)

In [ ]:
clustering_perfs = []

for i, hd in enumerate(ham_dists_ds):
    fig,ax = plt.subplots(figsize=(9,7))
    h1 = ax.imshow(hd, aspect='auto')
    fig.colorbar(h1, ax=ax)
    plt.show()
    
    # Cluster the distance matrix
    clusters = SpectralClustering(n_clusters=N_EMBRYOS,
                                  assign_labels="kmeans", affinity='precomputed').fit(np.exp(-hd**2))
    rand_score = adjusted_rand_score(true_clusterings[i],clusters.labels_)
    
    clustering_perfs.append(rand_score)

## Create Clustering Performance Figure

In [ ]:
clustering_perfs, avg_umi

In [ ]:
# Plot performance vs. UMI per cell
plt.figure(figsize=(10, 10), dpi=600)

plt.plot(avg_umi, clustering_perfs, '-o', lw=5, markersize=14)

plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=24)
plt.xticks(range(250, 2500, 500), fontsize=24, rotation=45)

plt.ylabel('Clustering Performance (ARI)', fontsize=28, labelpad=10)
plt.xlabel('Average UMI Counts per Cell', fontsize=28, labelpad=10)
plt.savefig(f'{N_EMBRYOS}_embryos.pdf')